In [1]:
PROJECT_ID = 'razekmh-angola'

In [2]:
# install libraries
! pip install -q earthengine-api geemap


In [3]:
# import libraries
import ee
import geemap
#geemap.update_package()


In [4]:
# authenticate to Earth Engine
ee.Authenticate()

True

In [5]:
# initialize the project    
ee.Initialize(project=PROJECT_ID)

In [6]:
# make a map
Map = geemap.Map(center=[-12.0, 18.0], zoom=8)

In [7]:
# add outline of Angola to the map

countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
angola_outline = countries.filter(ee.Filter.eq('ADM0_NAME', 'Angola'))
Map.addLayer(angola_outline, {}, 'Angola')


In [8]:
landsat_8_image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
    .filterBounds(angola_outline) \
        .filterDate('2020-01-01', '2024-08-01') \
            .filterMetadata('CLOUD_COVER', 'less_than', 10)

# Applies scaling factors. ref # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#colab-python
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

landsat_8_image_collection = landsat_8_image_collection.map(apply_scale_factors)

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

In [9]:
# display the map
Map.addLayer(landsat_8_image_collection, visualization, 'Landsat 8')
Map

Map(center=[-12.0, 18.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [10]:
landsat_9_image_collection = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")\
    .filterBounds(angola_outline) \
        .filterDate('2021-01-01', '2024-08-01') \
            .filterMetadata('CLOUD_COVER', 'less_than', 10)

# Applies scaling factors. ref # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2#colab-python
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

landsat_9_image_collection = landsat_9_image_collection.map(apply_scale_factors)

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

In [11]:
# display the map
Map.addLayer(landsat_9_image_collection, visualization, 'Landsat 9')
Map

Map(center=[-12.0, 18.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [12]:
vis_params = {'bands': ['SR_B7', 'SR_B5', 'SR_B2'], 'gamma': 1.0, 'min': 0.0, 'max': 0.3}

In [13]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)
# Sentinal2 collection 

sentinal2_collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
            .filterBounds(angola_outline)\
                .filterDate('2020-06-01', '2024-08-1')\
    # Pre-filter to get less cloudy granules.
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}



In [16]:
# display the map

Map.addLayer(sentinal2_collection.mean(), visualization, 'Sentinal2')
Map

Map(bottom=34860.0, center=[-9.79567758282973, 17.932434082031254], controls=(WidgetControl(options=['position…

In [22]:
#ASTER collect
aster_collection = (
    ee.ImageCollection("ASTER/AST_L1T_003")\
    .filterBounds(angola_outline)\
      
        .filterDate('2018-01-01', '2024-08-01') 
)
visualization = {
    'bands': ['B3N', 'B02', 'B01'],
    'min': 0.0,
    'max': 255.0,
}
# display the map
Map.addLayer(aster_collection, visualization, 'ASTER')
Map

Map(bottom=8756.0, center=[-5.7908968128719565, 20.775146484375004], controls=(WidgetControl(options=['positio…

In [23]:

import geodemo
import os


In [24]:
in_shp = '/Angola_Project Cods/Bacia_de_Kassa_ExportFeature.shp'
out_geojson='/Angola_Project Cods/Kassanje bound_FeaturesToJSO.geojson'


In [28]:
#geodemo.shp_to_geojson(in_shp, out_geojson=None)
#m=geodemo.Map(center=[-12.0, 18.0], zoom=8)
#m
# or as this code for add shapefile
Map.addLayer(out_geojson, visualization, 'Kassanje')
Map

Map(bottom=8878.0, center=[-8.450638800331001, 21.170654296875004], controls=(WidgetControl(options=['position…

In [ ]:
# o import libraries
import ee
import json

# initialize earth engine client
#ee.Initialize()

# load your gemotry data (which should be in GeoJson file)
with open("Kassanje bound_FeaturesToJSO.geojson") as f:
    geojson = json.load(f)

# construct a FeatureCollection object from the json object
fc = ee.image(geojson)

In [ ]:
# Load 2012 MODIS land cover and select the IGBP classification.
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define a palette for the 18 distinct land cover classes.
igbp_palette = [
    'aec3d4',  # water
    '152106',
    '225129',
    '369b47',
    '30eb5b',
    '387242',  # forest
    '6a2325',
    'c3aa69',
    'b76031',
    'd9903d',
    '91af40',  # shrub, grass
    '111149',  # wetlands
    'cdb33b',  # croplands
    'cc0013',  # urban
    '33280d',  # crop mosaic
    'd7cdcc',  # snow and ice
    'f7e084',  # barren
    '6f6f6f',  # tundra
]

# Define a map centered on the Angola.
map_palette = geemap.Map(center=[-12.0, 18.0], zoom=8)

# Add the image layer to the map and display it. Specify the min and max labels
# and the color palette matching the labels.
map_palette.add_layer(
    cover, {'min': 0, 'max': 17, 'palette': igbp_palette}, 'IGBP classes'
)
display(map_palette)

In [ ]:
js_snippet= """
var dataset = ee.ImageCollection('ASTER/AST_L1T_003')
                  .filter(ee.Filter.date('2018-01-01', '2018-08-15'));
var falseColor = dataset.select(['B3N', 'B02', 'B01']);
var falseColorVis = {
  min: 0.0,
  max: 255.0,
};
Map.setCenter(-122.0272, 39.6734, 11);
Map.addLayer(falseColor.median(), falseColorVis, 'False Color');

"""

In [ ]:
geemap.js_snippet_to_py(js_snippet,add_new_cell=True, import_ee=True, import_geemap=True,show_map=True)